# 郵遞區號資料更新器

Author: YuShan Lin(Erin Lin) erinylin@gmail.com

資料來源：[中華郵政全球資訊網 / 下載專區 / 郵務一覽](https://www.post.gov.tw/post/internet/Download/index.jsp?ID=220399)

取用資料為：4.2 3+2碼郵遞區號Excel檔 108/04(zip檔), 路徑http://download.post.gov.tw/post/download/Zip32_10804.zip

In [1]:
import pandas as pd
import numpy as np

In [2]:
from zipfile import ZipFile
import requests
import glob

In [3]:
path = 'http://download.post.gov.tw/post/download/Zip32_10804.zip'
filename = path.split('/')[-1]

## 下載資料

### 下載目標 .zip 並存到本機端

In [4]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}  
filedata = requests.get(path, headers=headers)  

with open(filename, 'wb') as f:  
    f.write(filedata.content)

### 解壓縮並取得檔案清單

In [5]:
zf = ZipFile(filename, 'r')
zf.extractall('_temp')
zf.close()

In [6]:
files = [f for f in glob.glob('_temp/' + "**/*.xls", recursive=True)]
files

['_temp/3+2╢l╗╝░╧╕╣└╔.xls']

### pandas 讀入資料進行預處理

In [26]:
df = pd.read_excel(files[0])

In [27]:
df.head()

,郵遞區號,縣市名稱,鄉鎮市區,原始路名,投遞範圍
0,10058,臺北市,中正區,八德路１段,全
1,10079,臺北市,中正區,三元街,單全
2,10070,臺北市,中正區,三元街,雙 48號以下
3,10079,臺北市,中正區,三元街,雙 50號以上
4,10068,臺北市,中正區,大埔街,單 15號以上


In [28]:
df.dtypes

郵遞區號     int64
縣市名稱    object
鄉鎮市區    object
原始路名    object
投遞範圍    object
dtype: object

In [29]:
# 資料只需要: 郵遞區號, 縣市名稱與鄉鎮市區
df = df.loc[:, df.columns[0:3]]

In [30]:
# 只保留前三碼郵遞區號
df['郵遞區號'] = df['郵遞區號'].astype(str).str[:3]

df.head()

,郵遞區號,縣市名稱,鄉鎮市區
0,100,臺北市,中正區
1,100,臺北市,中正區
2,100,臺北市,中正區
3,100,臺北市,中正區
4,100,臺北市,中正區


In [31]:
lists = []

# groupby code and 鄉鎮市區
for name, group in df.groupby(['郵遞區號', '鄉鎮市區']):
    # 取得 group 第一組資料, column 順序為 郵遞區號 縣市名稱 鄉鎮市區
    #print(group.iloc[0].values)
    lists.append(group.iloc[0].values)

# 建立新的 dataframe, 順便改一下 columns name
df2 = pd.DataFrame(lists, columns=('code', 'city', 'dist'))
df2.head()

,code,city,dist
0,100,臺北市,中正區
1,103,臺北市,大同區
2,104,臺北市,中山區
3,105,臺北市,松山區
4,106,臺北市,大安區


In [32]:
# 不打亂 index 取得城市順序
cities = []
# 將離島排到後面
island = ['金門縣', '連江縣', '澎湖縣']

for row in df2['city']:
    if row not in cities and row not in island:
        cities.append(row) 
        
cities = cities + island

In [33]:
# 組成程式需要的格式
result = dict()

for city in cities:
    city_df = df2.loc[ df2.city == city, ["dist", "code"]]
    result[city] = dict(zip( city_df['dist'], city_df['code'] ))
    
result

{'臺北市': {'中正區': '100',
  '大同區': '103',
  '中山區': '104',
  '松山區': '105',
  '大安區': '106',
  '萬華區': '108',
  '信義區': '110',
  '士林區': '111',
  '北投區': '112',
  '內湖區': '114',
  '南港區': '115',
  '文山區': '116'},
 '基隆市': {'仁愛區': '200',
  '信義區': '201',
  '中正區': '202',
  '中山區': '203',
  '安樂區': '204',
  '暖暖區': '205',
  '七堵區': '206'},
 '新北市': {'萬里區': '207',
  '金山區': '208',
  '板橋區': '220',
  '汐止區': '221',
  '深坑區': '222',
  '石碇區': '223',
  '瑞芳區': '224',
  '平溪區': '226',
  '雙溪區': '227',
  '貢寮區': '228',
  '新店區': '231',
  '坪林區': '232',
  '烏來區': '233',
  '永和區': '234',
  '中和區': '235',
  '土城區': '236',
  '三峽區': '237',
  '樹林區': '238',
  '鶯歌區': '239',
  '三重區': '241',
  '新莊區': '248',
  '泰山區': '243',
  '林口區': '244',
  '蘆洲區': '247',
  '五股區': '248',
  '八里區': '249',
  '淡水區': '251',
  '三芝區': '252',
  '石門區': '253'},
 '宜蘭縣': {'壯圍鄉': '263',
  '宜蘭市': '260',
  '頭城鎮': '261',
  '礁溪鄉': '262',
  '員山鄉': '264',
  '羅東鎮': '265',
  '三星鄉': '266',
  '大同鄉': '267',
  '五結鄉': '268',
  '冬山鄉': '269',
  '蘇澳鎮': '270',
  '南澳鄉': '272',
  '釣魚臺': 

In [34]:
output = """
//台灣郵遞區號 %s
var __tw_zipcode = %s;
"""

with open('../js/er.twzipcode.data.js', 'w') as f:  
    f.write(output % (path, str(result) ))

## Delete _temp files

直接使用 command line 處理

In [35]:
import os
import sys

if sys.platform.startswith('darwin'):
    '''
    MacOS
    '''
    os.system("rm -rf _temp")
    os.system("rm %s" % filename)
elif sys.platform.startswith('win32'):
    '''
    windows
    '''
    os.system('rmdir /Q /S _temp')
    os.system('del /f %s' % filename)
    